In [ ]:
import numpy as np
import scipy as sp
from scipy import fftpack

import pandas as pd

import os
import glob

import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

In [ ]:
path = os.getcwd() + '\\data\\'
extension = 'csv'

os.chdir(path)
titles = glob.glob('*.{}'.format(extension))
print(titles)

In [ ]:
datasets = []
for d in titles:
    data = pd.read_csv(path + d)
    if len(data.columns) == 1:
        data = pd.read_csv(path + d, sep=";")
    data['action'] = data['Stimulus'].apply(lambda x: x.replace(' ', '.').split('.')[0])
    datasets.append(data[data.columns[1:]])

In [ ]:
data = datasets[1]
print(list(data.columns))

## Frequency distribution

In [ ]:
sb.set_context("paper")
for c in data.columns[1:-1]:
    plt.figure()
    sb.violinplot(data[c], data.action)

## Curve similarity by eye

In [ ]:
def plot_confidence(data):
    avg = np.mean(data, axis = 0)
    sd = np.std(data, axis = 0)
    
    plt.plot(avg)
    plt.fill_between(list(range(len(avg))), avg - sd, avg + sd, alpha=.3, facecolor='red')

    
def pre_analysis(data, action, repetitions, wave, stimulus_len = None):
    dt = data[data.action == action].copy()
    dt = dt[wave]
    
    if stimulus_len is None:
        stimulus_len = int(dt.shape[0] / repetitions)
        
    datamatrix = np.zeros([repetitions, stimulus_len])
    for i in range(repetitions):
        datamatrix[i, :] = dt[i*stimulus_len:(i+1)*stimulus_len]
    
    plt.figure()
    plt.title(action)
    plot_confidence(datamatrix)

In [ ]:
wave = "AF3"
for action in set(data.action.values):
    pre_analysis(data, action, 30, wave)

## Signal Subsetting

In [ ]:
## Split the signals into small subsets with an overlap
def subset_data(data, ss_dim = None, ss_num = 10, overlap = .5, cut_smaller = True):
    if overlap > 1: # in case the value passed is a percentage
        overlap = float(overlap)/100
        
    if ss_dim is None: # either choose the dimension of the subsets or the number
        ss_dim = int(len(data)/(ss_num*overlap)) # by default it will divide the signal in 10 subsets
    
    subsets = []
    i = len(data) - 1
    while i >= 0:
        j = max(i - ss_dim, 0)
        subsets.append(data[j:i])
        i -= int(ss_dim * (1 - overlap))
    
    while cut_smaller and len(subsets[-1]) < ss_dim :
        subsets = subsets[:-1]
        
    return np.array(subsets)

def prepare_data(dataframe, ss_dim = 20, ss_num = 168, overlap = .5):
    dataset = {}
    for action in set(dataframe.action.values):
        a = []
        data = dataframe[dataframe.action == action].copy()
        for c in data.columns[1:-1]:
            a.append(np.array(subset_data(np.array(data[c]), ss_dim, ss_num, overlap, cut_smaller = True)))
        dataset[action] = [np.asmatrix(d).flatten() for d in np.transpose(np.array(a), [1, 2, 0])] # if you want a list of matrixes
        #dataset[action] = np.transpose(np.array(a), [1, 2, 0]) # if you want a tensor
    return dataset

In [ ]:
dataset = prepare_data(data)

In [ ]:
print(len(dataset["run"]), dataset['run'][0].shape)


## Clustering

In [ ]:
import sklearn.cluster as csr
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

from sklearn import metrics

from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler

In [ ]:
def make_pivot_table(Y, actions):
    pivot_table = np.zeros([len(actions), clust_num])
    for i, action in enumerate(actions):
        for cluster in range(clust_num):
            pivot_table[i, cluster] = len( Y[(Y.action == action) & (Y.cluster == cluster)] )
    return pivot_table

In [ ]:
X = np.zeros([0, 280])
labels = []
for t in dataset.keys():
    x = np.squeeze(dataset[t], axis = 1)
    labels += [t for i in range(int(x.shape[0]))]
    X = np.concatenate((X, x))
X = StandardScaler().fit_transform(X)

In [ ]:
clust_num = 7
ac = csr.AgglomerativeClustering(n_clusters = clust_num, compute_full_tree=True)
ac.fit(X)
clust_labels = ac.labels_

Y=pd.DataFrame(data = {'action': labels, 'cluster': clust_labels})

In [ ]:
actions = list(set(data.action.values))
pivot_table = make_pivot_table(Y, actions)
print(actions)
print(pivot_table)
print(sp.stats.chisquare(pivot_table, axis=0)[1])

In [ ]:
## Bruteforce parameter optimizer
dimensions = [10, 15, 20, 30, 50]
cluster_number = [4, 5, 6, 7, 8]

combination = []
for dim in dimensions:
    dataset = prepare_data(data, ss_dim = dim)

    X = np.zeros([0, 14*dim])
    labels = []
    for t in dataset.keys():
        x = np.squeeze(dataset[t], axis = 1)
        labels += [t for i in range(int(x.shape[0]))]
        X = np.concatenate((X, x))
    X = StandardScaler().fit_transform(X)
    for clust_num in cluster_number:
        ac = csr.AgglomerativeClustering(n_clusters = clust_num)
        ac.fit(X)
        clust_labels = ac.labels_
        Y = pd.DataFrame(data = {'action': labels, 'cluster': clust_labels})
        actions = list(set(Y.action.values))
        pivot_table = make_pivot_table(Y, actions)
        p_values = sp.stats.chisquare(pivot_table, axis=0)[1]
        combination.append([dim, clust_num, pivot_table, p_values])

In [ ]:
print actions
for combo in combination:
    for c in combo:
        print(c)
    print

# Fourier


In [ ]:
dt = data["AF3"]
dt = dt[data.Stimulus == "jump.png"].copy()

In [ ]:
FFT = abs(sp.fft(dt))
freqs = sp.fftpack.fftfreq(dt.size, 0.01)
plt.plot(freqs,20*sp.log10(FFT),'.')